In [33]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [32]:
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w


def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

## <center>Data Preparation<center>

Let's see on our data:

- `Id` - an Id that represents a (Store, Date) duple within the test set
- `Store` - a unique Id for each store
- `Sales` - the turnover for any given day (TARGET)
- `Customers` - the number of customers on a given day
- `Open` - an indicator for whether the store was open: 
    - 0 = closed
    - 1 = open
- `StateHoliday` - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. 
  - a = public holiday
  - b = Easter holiday
  - c = Christmas
  - 0 = None
- `SchoolHoliday` - indicates if the (Store, Date) was affected by the closure of public schools
- `StoreType` - differentiates between 4 different store models: a, b, c, d
- `Assortment` - describes an assortment level: 
    - a = basic
    - b = extra
    - c = extended
- `CompetitionDistance` - distance in meters to the nearest competitor store
- `CompetitionOpenSince`[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
- `Promo` - indicates whether a store is running a promo on that day
- `Promo2` - Promo2 is a continuing and consecutive promotion for some stores:
    - 0 = store is not participating
    - 1 = store is participating
- `Promo2Since`[Year/Week] - describes the year and calendar week when the store started participating in Promo2
- `PromoInterval` - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [12]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_store = pd.read_csv("data/store.csv")

In [4]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [6]:
df_store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [13]:
df_store = df_store[["Store","StoreType", "Assortment", "CompetitionDistance","Promo2"]]
df_store = df_store.set_index("Store")
df_store.CompetitionDistance = df_store.CompetitionDistance.fillna(df_store.CompetitionDistance.max())
df_store.head()

,StoreType,Assortment,CompetitionDistance,Promo2
Store,,,,
1,c,a,1270.0,0
2,a,a,570.0,1
3,a,a,14130.0,1
4,c,c,620.0,0
5,a,a,29910.0,0


In [14]:
def data_preparation(df):
    df = df.join(df_store, on="Store")
    df.fillna(0, inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].map(lambda x: x.year)
    df['Month'] = df['Date'].map(lambda x: x.month)
    df['dayOfWeek'] = df['Date'].map(lambda x: x.dayofweek)
    
    df.loc[df.StateHoliday == '0', 'StateHoliday'] = 0
    holidays = pd.get_dummies(df['StateHoliday'], prefix='Hol')
    store_type = pd.get_dummies(df['StoreType'], prefix='Type')
    assort = pd.get_dummies(df['Assortment'], prefix='Assort')
    df = pd.concat((df, holidays, store_type, assort), axis=1)  
    
    df = df.drop(['Store','Date','DayOfWeek','Assortment','StateHoliday','StoreType'], axis=1)
    return df

In [15]:
train = data_preparation(df_train)
train = train[train.Open == 1]
test = data_preparation(df_test)

### <center>Visualization<center>

In [16]:
X = train.drop(['Sales','Customers','Open','Hol_b','Hol_c'],axis=1)
y = train.Sales.values

In [29]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y , test_size=0.7)

In [30]:
rf = RandomForestRegressor(n_jobs=-1, criterion='mse')
%time rf.fit(X_train, y_train)

CPU times: user 19.3 s, sys: 119 ms, total: 19.4 s
Wall time: 6.26 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [31]:
pred = rf.predict(X_valid)
rmspe(pred, y_valid)

0.3129121305136074

### <center>Write to csv file<center>

In [18]:
model = RandomForestRegressor(min_samples_leaf=2, max_depth=30, n_estimators=30)
%time model.fit(X, y)

CPU times: user 1min 40s, sys: 375 ms, total: 1min 40s
Wall time: 1min 40s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [19]:
y = model.predict(test.values[:, 2:])

df = pd.DataFrame([])
df['Sales'] = y
df['Sales'][df_test.Open == 0] = 0
df = df.set_index(df_test.Id)
pd.DataFrame.to_csv(df, 'ans1.csv')
df.head()

,Sales
Id,
1,7330.503135
2,7830.815794
3,9914.799206
4,8281.157857
5,7875.188901


In [ ]:
TO	0.19
1	6756.614163
2	7777.839524
3	10181.776032
4	8324.168333
5	7936.701915